In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [234]:
# 테이블 형식의 데이터를 처리하는 pandas 라이브러리를 임포트
import pandas as pd

In [508]:
# train.csv, test.csv 파일을 DataFrame 형태로 읽어옵니다.
train_df = pd.read_csv('/content/drive/MyDrive/datathon/dataset/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/datathon/dataset/test.csv')
print('train shape: ', train_df.shape)
print('test shape: ', test_df.shape)
train_df.info()

train shape:  (9375, 10)
test shape:  (4018, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9375 entries, 0 to 9374
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      9375 non-null   float64
 1   gender                   9375 non-null   object 
 2   height_cm                9375 non-null   float64
 3   diastolic                9375 non-null   float64
 4   systolic                 9375 non-null   float64
 5   gripForce                9375 non-null   float64
 6   sit and bend forward_cm  9375 non-null   float64
 7   sit-ups counts           9375 non-null   float64
 8   broad jump_cm            9375 non-null   float64
 9   body fat_%               9375 non-null   float64
dtypes: float64(9), object(1)
memory usage: 732.5+ KB


In [509]:
# train_df에서 체지방율 열을 pop 하여 y_train에 저장합니다.
y_train = train_df.pop('body fat_%')

In [510]:
# 베이스라인 코드에서는 성별(카테고리 데이터)을 제외하고 수치 데이터만 활용해 봅니다.
# 학습 및 평가 셋에 각각 성별(gender)을 제거해줍니다.
train_df['gender'] = train_df['gender'].map({'M':0, 'F':1})
test_df['gender'] = test_df['gender'].map({'M':0, 'F':1})

#정규화를 위해서 gender field 분리
train_gtemp = train_df.pop('gender')
test_gtemp = test_df.pop('gender')


In [511]:
# 8개 컬럼의 수치 데이터를 정규화 하여 학습 가능한 형태로 재가공 합니다.
# 다른 정규화 방식을 찾아서 적용해보고 정규화 방식에 따라 결과가 어떻게 달라지는지 확인해봅니다.
from sklearn.preprocessing import MinMaxScaler 

# 학습 및 평가 데이터를 DataFrame 형태로 입력받아 정규화된 데이터로 변환하는 함수 
def scale(x_train, x_test):
    columns = x_train.columns

    scaler = MinMaxScaler()

    # 학습 데이터의 age ~ broad jump_cm 열에 해당하는 값을 가져와 피팅 시킵니다.
    scaler.fit(x_train)

    # 피팅된 scaler를 기준으로 학습데이터를 정규화합니다.
    x_train = scaler.transform(x_train)
    # 평가 데이터도 마찬가지로 학습 데이터로 피팅된 scaler를 기준으로 정규화를 진행합니다.
    x_test = scaler.transform(x_test)

    # DataFrame 형태로 변환하여 반환합니다.
    x_train = pd.DataFrame(columns=columns, data=x_train)
    x_test = pd.DataFrame(columns=columns, data=x_test)

    return x_train, x_test

# 학습 데이터, 평가 데이터를 정의한 scale 함수에 전달하여 정규화된 학습 및 평가 데이터를 x_train, x_test에 저장합니다.
x_train, x_test = scale(train_df, test_df)

In [512]:
x_train['gender'] = train_gtemp
x_test['gender'] = test_gtemp

x_train['gender'] = x_train['gender'].astype('bool')
x_test['gender'] = x_test['gender'].astype('bool')

In [513]:
# 학습을 수행하기 전에 모델 학습이 제대로 진행되었는지 확인하기 위해 검증셋을 분리해 놓습니다.
# 검증셋은 학습에 사용하지 않고 모델을 평가하기 위해 사용됩니다.
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)

(7500, 9) (7500,)
(1875, 9) (1875,)


In [518]:
#모델 추가
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [629]:
#모델 객체 생성
model_dic = {
    "Linear" : LinearRegression(),
    "SVM" : svm.SVR(kernel='rbf', C = 6.25, gamma=1.1),
    "RandomForest" : RandomForestRegressor(),
    "xgboost" : XGBRegressor(),
}

#"xgboost" : XGBRegressor() 

In [630]:
model_p = {}

for key in model_dic.keys() :
  print(f'{key} : training')
  model_dic[key].fit(x_train, y_train)

  print(f'{key} : predict')
  model_p[key] = model_dic[key].predict(x_valid)

Linear : training
Linear : predict
SVM : training
SVM : predict
RandomForest : training
RandomForest : predict
xgboost : training
[07:25:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
xgboost : predict


In [631]:
#결과를 데이터프레임으로
model_pdf = pd.DataFrame(model_p)
model_pdf['valid'] = list(y_valid)

# 모델평가

In [525]:
from sklearn.metrics import mean_absolute_error
import numpy as np

In [632]:
model_mae = {}

for key in model_dic.keys() :
  mae = mean_absolute_error(model_pdf['valid'], model_p[key])
  model_mae[key] = mae

In [633]:
model_mae

{'Linear': 3.82428666325485,
 'RandomForest': 3.8441143640906668,
 'SVM': 3.7405188124252833,
 'xgboost': 3.777044651039876}

# 평가 데이터 예측 및 제출 파일 생성

In [635]:
# 검증셋 예측과 동일하게 평가 데이터 예측을 진행합니다.
preds = model_dic['SVM'].predict(x_test)
print(preds.shape)

(4018,)


In [636]:
# 제출파일 예시 파일을 읽어와 predict 열에 평가셋 예측값을 기록하여 저장합니다.
submit = pd.read_csv('/content/drive/MyDrive/datathon/dataset/submit_sample.csv')
submit['predict'] = preds

# 제출 파일 확인
submit

,predict
0,23.320110
1,22.345531
2,16.025266
3,19.500507
4,36.720476
...,...
4013,19.987955
4014,18.787552
4015,26.750057
4016,17.904372


In [637]:
# 저장
submit.to_csv('/content/drive/MyDrive/datathon/dataset/submit.csv', index=False)